
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment 5 - RL <br>
<font color=696880 size=4>
Amirhossein Abedi


____

# Personal Data

In [2]:
# Set your student number and name
student_number = '401106096'
Name = 'Radin'
Last_Name = 'Shahdaei'

# <center>تاکسی</center>

<div dir="rtl" style="font-size:18px">
در این سوال می‌خواهیم با محیط Taxi در gymnasium کار کنیم.
<br>
در اینجا در نقش یک تاکسی هستیم که باید یک مسافر را از یک مکان مشخص برداریم و در مقصد موردنظر او پیاده کنیم. توضیحات تکمیلی درمورد این محیط را می‌توانید در  <a href="https://gymnasium.farama.org/environments/toy_text/taxi/">Taxi</a>  پیدا کنید.

![Taxi](taxi.png)
</div>

<div dir="rtl" style="font-size:18px">
در این سوال می‌خواهیم با پیاده‌سازی الگوریتم value iteration بهترین پاسخ در هر حالت را پیدا کنیم. هر حالت در این بازی با یک عدد بین ۰ تا ۴۹۹ نشان داده می‌شود. که فرمول آن به صورت زیر است
<br><br>

### <center dir="ltr">state = ((5 × row + col) × 5 + passenger_location) × 4 + destination</center>

<br><br>
مقادیر row و col که مشخص است و عددی بین ۰ تا ۴ است. مقدار passenger_location بین ۰ تا ۴ می‌باشد که هرکدام از اعداد نشان می‌دهد که مسافر در کدام پایانه قرار دارد. پایانه‌ها مکان‌هایی مشخص درون بازی هستند که می‌تواند با استفاده از تابع env.render آن‌ها را مشاهده کنید. ۴ پایانه به رنگ‌های قرمز، سبز، زرد، آبی درون بازی قرار دارند که مقادیر ۰ تا ۳ به ترتیب همین پایانه‌ها را نشان می‌دهد. و در نهایت مقدار ۴ برای passenger_location به این معنا است که مسافر در تاکسی قرار دارد.
<br><br>
مقدار destination نشان می‌دهد که مسافر می‌خواهد در کدام پایانه پیاده شود. نگاشت عدد و رنگ‌ها مانند بخش passenger_location می‌باشد و فقط مقادیر ۰ تا ۳ را به خود می‌گیرد.
<br><br>
با هر بار اجرای تابع reset و step بر روی env ، مقدار اولی که به شما برگردانده می‌شود مقدار state است. در ادامه به action ها می‌پردازیم. در کل ۶ اکشن مختلف می‌توانیم اجرا کنیم. اکشن ۰ تا ۳ به ترتیب مربوط به حرکات پایین، بالا، راست و چپ است. حرکت ۴ مربوط به برداشتن مسافر و حرکت ۵ پیاده کردن مسافر است. استفاده از حرکت ۴ و ۵ هنگامی که مجاز نباشند، ریوارد منفی زیادی خواهد داشت.
</div>

In [ ]:
!pip install gymnasium

In [4]:
import gymnasium as gym
env = gym.make('Taxi-v3', render_mode='ansi')

<div dir="rtl" style="font-size:18px">
خروجی تابع reset شامل یک ۲ تایی است. اولین خروجی آن یک عدد است که نشان‌دهنده حالت اولیه می‌باشد و خروجی دوم یک دیکشنری است که مقدار action_mask شامل تمام اکشن‌هایی است که در این حالت می‌توانید انجام دهید.
</div>

In [5]:
env.reset()

(412, {'prob': 1.0, 'action_mask': array([0, 1, 0, 0, 0, 0], dtype=int8)})

<div dir="rtl" style="font-size:18px">
خروجی تابع step هم تا حدود زیادی شبیه به reset می باشد. با این تفاوت که بعد از خروجی حالت، یک عدد و ۲ بولین به شما می‌دهد که عدد اول مقدار reward و بولین اول تمام شدن بازی را مشخص می‌کند.
</div>

In [6]:
env.step(0)

(412,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([0, 1, 0, 0, 0, 0], dtype=int8)})

<div dir="rtl" style="font-size:18px">
در ادامه کلاس VI را پیاده‌سازی کنید که با گرفتن discount factor الگوریتم value iteration را پیاده‌سازی می‌کند.
</div>

In [13]:
import numpy as np

class VI():
    def __init__(self, gamma: float = 0.95):
        self.gamma = gamma
        self.vals = np.zeros(500)
        NUM_LOOPS = 10000
        self.train_value_iteration(NUM_LOOPS)

    def train_value_iteration(self, num_loops: int):
        """Train the value iteration algorithm for a specified number of loops."""
        for _ in range(num_loops):
            vals_prime = np.copy(self.vals)
            for i in range(500):
                vals_prime[i] = self.calculate_value(i)
            self.vals = vals_prime
            self.set_terminal_values()

    def calculate_value(self, state: int) -> float:
        """Calculate the value of a given state by considering all possible actions."""
        values = [self.gamma * self.vals[self.T(state, action)] + self.R(state, action) for action in range(6)]
        return max(values)

    def set_terminal_values(self):
        """Set predefined terminal values for specific states."""
        terminal_states = [
            self._state_to_num(0, 0, 4, 0),
            self._state_to_num(0, 4, 4, 1),
            self._state_to_num(4, 0, 4, 2),
            self._state_to_num(4, 3, 4, 3)
        ]
        for state in terminal_states:
            self.vals[state] = 1000

    def policy(self, state: int) -> int:
        """Determine the best action to take in a given state based on the current value function."""
        values = [self.vals[self.T(state, action)] + self.R(state, action) for action in range(6)]
        return int(np.argmax(values))

    def T(self, state: int, action: int) -> int:
        """Transition function: returns the next state given a current state and action."""
        row, col, passenger_loc, dest = self._num_to_state(state)
        if action == 4:
            return self.handle_pickup(row, col, passenger_loc, dest, state)
        if action == 5:
            return state
        return self.handle_move(row, col, passenger_loc, dest, action, state)

    def handle_pickup(self, row: int, col: int, passenger_loc: int, dest: int, state: int) -> int:
        """Handle the pickup action."""
        if self._correct_pickup(row, col, passenger_loc):
            return self._state_to_num(row, col, 4, dest)
        return state

    def handle_move(self, row: int, col: int, passenger_loc: int, dest: int, action: int, state: int) -> int:
        """Handle the movement actions (north, south, east, west)."""
        if action == 0:
            return self.move_south(row, col, passenger_loc, dest, state)
        if action == 1:
            return self.move_north(row, col, passenger_loc, dest, state)
        if action == 2:
            return self.move_east(row, col, passenger_loc, dest, state)
        if action == 3:
            return self.move_west(row, col, passenger_loc, dest, state)
        return state

    def move_south(self, row: int, col: int, passenger_loc: int, dest: int, state: int) -> int:
        """Move south (down) if not at the boundary."""
        if row == 4:
            return state
        return self._state_to_num(row + 1, col, passenger_loc, dest)

    def move_north(self, row: int, col: int, passenger_loc: int, dest: int, state: int) -> int:
        """Move north (up) if not at the boundary."""
        if row == 0:
            return state
        return self._state_to_num(row - 1, col, passenger_loc, dest)

    def move_east(self, row: int, col: int, passenger_loc: int, dest: int, state: int) -> int:
        """Move east (right) if not at the boundary or blocked."""
        if (row, col) in self.get_east_blocked_cells():
            return state
        return self._state_to_num(row, col + 1, passenger_loc, dest)

    def move_west(self, row: int, col: int, passenger_loc: int, dest: int, state: int) -> int:
        """Move west (left) if not at the boundary or blocked."""
        if (row, col) in self.get_west_blocked_cells():
            return state
        return self._state_to_num(row, col - 1, passenger_loc, dest)

    def get_east_blocked_cells(self):
        """Return the list of cells where movement to the east is blocked."""
        return [(0, 1), (1, 1), (3, 0), (4, 0), (3, 2), (4, 2), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]

    def get_west_blocked_cells(self):
        """Return the list of cells where movement to the west is blocked."""
        return [(3, 3), (4, 3), (0, 2), (1, 2), (3, 1), (4, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]

    def R(self, state: int, action: int) -> int:
        """Reward function: returns the reward for a given state and action."""
        row, col, passenger_loc, dest = self._num_to_state(state)
        if action == 4:
            return self.pickup_reward(row, col, passenger_loc)
        if action == 5:
            return self.dropoff_reward(row, col, passenger_loc, dest)
        return -1

    def pickup_reward(self, row: int, col: int, passenger_loc: int) -> int:
        """Calculate the reward for picking up a passenger."""
        if self._correct_pickup(row, col, passenger_loc):
            return -1
        return -10

    def dropoff_reward(self, row: int, col: int, passenger_loc: int, dest: int) -> int:
        """Calculate the reward for dropping off a passenger."""
        if self._correct_drop(row, col, passenger_loc, dest):
            return 20
        return -10

    def _correct_pickup(self, row: int, col: int, passenger_loc: int) -> bool:
        """Check if the pickup location is correct."""
        pickup_locations = {0: (0, 0), 1: (0, 4), 2: (4, 0), 3: (4, 3)}
        return (row, col) == pickup_locations.get(passenger_loc, (-1, -1))

    def _correct_drop(self, row: int, col: int, passenger_loc: int, dest: int) -> bool:
        """Check if the dropoff location is correct."""
        dropoff_locations = {0: (0, 0), 1: (0, 4), 2: (4, 0), 3: (4, 3)}
        return passenger_loc == 4 and (row, col) == dropoff_locations.get(dest, (-1, -1))

    def _num_to_state(self, num: int):
        """Convert a state number to the corresponding (row, col, passenger_loc, dest) tuple."""
        dest = num % 4
        num //= 4
        passenger_loc = num % 5
        num //= 5
        col = num % 5
        num //= 5
        return num, col, passenger_loc, dest

    def _state_to_num(self, row: int, col: int, passenger_loc: int, dest: int) -> int:
        """Convert a (row, col, passenger_loc, dest) tuple to the corresponding state number."""
        ret = row
        ret = 5 * ret + col
        ret = 5 * ret + passenger_loc
        ret = 4 * ret + dest
        return ret

agent = VI()


<div dir="rtl" style="font-size:18px">
۲ سلول بعدی را اجرا کنید. در هر نقشه نشان داده شده است که با برقرار بودن شرایط به خصوص، تاکسی به کدام سمت حرکت می‌کند. در انتها در چند خط منطقی عمل کردن مدل خود را بررسی کنید.
</div>

In [14]:
terminal = {0: 'R', 1: 'G', 2: 'Y', 3: 'B'}
arrows = {0: '↓', 1: '↑', 2: '→', 3: '←'}
for passenger_loc in range(4):
    print(f'Passenger in {terminal[passenger_loc]}')
    print('+---------+')
    for i in range(5):
        print('|', end='')
        for j in range(5):
            if not j == 0:
                if (i, j) in [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]:
                    print('|', end='')
                else:
                    print(':', end='')
            if (i, j) in [(0, 0), (0, 4), (4, 0), (4, 3)]:
                print(terminal[[(0, 0), (0, 4), (4, 0), (4, 3)].index((i, j))], end='')
            else:
                try:
                    print(arrows[agent.policy(agent._state_to_num(i, j, passenger_loc, 1))], end='')
                except:
                    print('⤬', end='')
        print('|')
    print('+---------+\n')



Passenger in R
+---------+
|R:←|↓:↓:G|
|↑:↑|↓:↓:↓|
|↑:↑:←:←:←|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger in G
+---------+
|R:↓|→:→:G|
|↓:↓|↑:↑:↑|
|→:→:↑:↑:↑|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger in Y
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|↓:←:←:←:←|
|↓|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger in B
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|→:→:→:↓:↓|
|↑|↑:↑|↓:↓|
|Y|↑:↑|B:←|
+---------+



In [15]:
terminal = {0: 'R', 1: 'G', 2: 'Y', 3: 'B'}
arrows = {0: '↓', 1: '↑', 2: '→', 3: '←'}
for dest in range(4):
    print(f'Passenger on board, destination in {terminal[dest]}')
    print('+---------+')
    for i in range(5):
        print('|', end='')
        for j in range(5):
            if not j == 0:
                if (i, j) in [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]:
                    print('|', end='')
                else:
                    print(':', end='')
            if (i, j) in [(0, 0), (0, 4), (4, 0), (4, 3)]:
                print(terminal[[(0, 0), (0, 4), (4, 0), (4, 3)].index((i, j))], end='')
            else:
                try:
                    print(arrows[agent.policy(agent._state_to_num(i, j, 4, dest))], end='')
                except:
                    print('⤬', end='')
        print('|')
    print('+---------+\n')

Passenger on board, destination in R
+---------+
|R:←|↓:↓:G|
|↑:↑|↓:↓:↓|
|↑:↑:←:←:←|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger on board, destination in G
+---------+
|R:↓|→:→:G|
|↓:↓|↑:↑:↑|
|→:→:↑:↑:↑|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger on board, destination in Y
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|↓:←:←:←:←|
|↓|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger on board, destination in B
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|→:→:→:↓:↓|
|↑|↑:↑|↓:↓|
|Y|↑:↑|B:←|
+---------+



<div dir="rtl" style="font-size:18px">
در تحلیل نقشه‌ها مشخص است که مدل تاکسی برای هر
موقعیت مسافر (R، G، Y، B) حرکت بهینه را انتخاب می‌کند. به عنوان مثال، وقتی مسافر در موقعیت R است، تاکسی به سمت چپ حرکت می‌کند و وقتی مسافر در موقعیت G است، تاکسی به سمت راست حرکت می‌کند. این روند در تمامی نقشه‌ها مشاهده می‌شود که نشان می‌دهد مدل به درستی موقعیت مسافر را شناسایی کرده و به سمت آن حرکت می‌کند. پس از سوار کردن مسافر، مدل مسیر بهینه را برای رسیدن به مقصد مشخص شده انتخاب می‌کند. به عنوان نمونه، زمانی که مقصد مسافر R است، تاکسی به سمت چپ حرکت می‌کند و زمانی که مقصد G است، تاکسی به سمت راست حرکت می‌کند.
<div>

<div dir="rtl" style="font-size:18px">
عملکرد مدل نشان می‌دهد که الگوریتم ارزش‌یابی (value iteration) به درستی پاداش‌ها و ارزش‌های هر حالت و عمل را محاسبه کرده است. این الگوریتم به مدل اجازه می‌دهد تا برای هر وضعیت و عملی که انجام می‌دهد، ارزش بیشتری را انتخاب کند و به سمت آن حرکت کند. در نتیجه، مدل به صورت منطقی و بهینه مسافر را به مقصد می‌رساند. این رفتار نشان‌دهنده این است که مدل با توجه به شرایط و پاداش‌های مشخص شده، به بهترین نحو ممکن تصمیم‌گیری می‌کند و مسافر را در کمترین زمان ممکن به مقصد می‌رساند.
<div>